# **Modelo de Machine Learning** 

### **Importamos las liberías a utilizar**

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import joblib
from datetime import datetime
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.model_selection import learning_curve
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

### **Cargamos nuestro dataset**

In [2]:
df = pd.read_parquet("Datasets/3-Normalizados/taxis_2023.parquet")

In [3]:
df.columns

Index(['pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'PULocationID', 'DOLocationID', 'total_amount',
       'service_type', 'pickup_borough', 'dropoff_borough'],
      dtype='object')

### **Tratamos los datos que se utilizarán en el modelo**

In [4]:
# Creamos una funcion que agregue una nueva columna llamada pBoroughID que contenga el ID del borough usando la columna pickup_borough de la
# siguiente manera: 1 si es Manhattan, 2 si es Brooklyn, 3 si es Queens, 4 si es Bronx, 5 si es Staten Island y 6 si EWR
def addBoroughID(df):
    df['pBoroughID'] = df['pickup_borough'].apply(lambda x: 1 if x == 'Manhattan' else (2 if x == 'Brooklyn'
    else (3 if x == 'Queens' else (4 if x == 'Bronx' else (5 if x == 'Staten Island' else 6)))))
    return df

In [5]:
# Aplicamos la funcion
df = addBoroughID(df)
df.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount,service_type,pickup_borough,dropoff_borough,pBoroughID
0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,161,141,14.30,yellow,Manhattan,Manhattan,1
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,43,237,20.90,yellow,Manhattan,Manhattan,1
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,48,238,49.90,yellow,Manhattan,Manhattan,1
3,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,107,79,22.96,yellow,Manhattan,Manhattan,1
4,2023-01-01 00:50:34,2023-01-01 01:02:52,1.0,1.84,161,137,37.80,yellow,Manhattan,Manhattan,1


In [6]:
# Eliminamos las columnas que no será necesarias para el modelo
df = df.drop(['PULocationID', 'DOLocationID'], axis=1)
df.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,total_amount,service_type,pickup_borough,dropoff_borough,pBoroughID
0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,14.30,yellow,Manhattan,Manhattan,1
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,20.90,yellow,Manhattan,Manhattan,1
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,49.90,yellow,Manhattan,Manhattan,1
3,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,22.96,yellow,Manhattan,Manhattan,1
4,2023-01-01 00:50:34,2023-01-01 01:02:52,1.0,1.84,37.80,yellow,Manhattan,Manhattan,1


#### **Para un calculo adecuado de la demanda por hora, procedemos a hacer unas conversiones**

In [7]:
# Obtenemos mes, día de la semana, hora y día de la semana
df['month'] = df['pickup_datetime'].dt.month
df['dayofweek'] = df['pickup_datetime'].dt.dayofweek
df['hour'] = df['pickup_datetime'].dt.hour

In [8]:
# Convertimos la columna 'pickup_datetime' a formato datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Creamos una nueva columna para la fecha y hora completa
df['datetime_pickup'] = df['pickup_datetime'] + pd.to_timedelta(df['pickup_datetime'].dt.hour, unit='H')

# Agrupamos los datos por hora y contamos la cantidad de viajes
demand_by_hour = df.groupby(df['pickup_datetime'].dt.hour)['passenger_count'].count()

In [9]:
# Subconjunto de las columnas que queremos analizar
columnas = [
    'hour',
    'pBoroughID',
    'dayofweek',
    'month',
    'trip_distance'
]

# Creamos un DataFrame que contenga solo estas columnas
subset = df[columnas]

# Calculamos la matriz de correlación
correlation_matrix = subset.corr()

# Crear el mapa de calor con Plotly Express
fig = px.imshow(
    correlation_matrix,
    color_continuous_scale='Viridis',
    labels=dict(x="Variables", y="Variables", color="Correlación"),
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    width=800,
    height=800
)

# Configuramos el diseño del gráfico
fig.update_layout(
    title='Matriz de Correlación',
)

# Mostramos el gráfico
fig.show()

In [10]:
# Agrupamos por localización, hora y día de la semana
df_ml = df.groupby(['hour', 'pBoroughID', 'dayofweek']).size().reset_index(name='demand')

In [11]:
#Creamos el porcentaje de Demanda
df_ml['demand'] = df_ml['demand'].apply(lambda x :  (x / df_ml['demand'].max()))
df_ml['demand'] = round(df_ml['demand']*100,2)

## **Iniciamos la búsqueda del modelo** 

#### **Planteamos el modelo de predicción de demanda, en base a la hora, locación y día de la semana**

In [12]:
# Seleccionamos las características (X) y la variable objetivo (y)
features = ['pBoroughID', 'dayofweek', 'hour']
target = 'demand'

#### **Planteamos las variables dependientes e independientes**

In [13]:
X = df_ml[features]
y = df_ml[target]

## **Random Forest**

In [14]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializamos y entrenamos el modelo RandomForestRegressor con parámetros ajustados
rf_model = RandomForestRegressor(
    n_estimators=200,   # Número de árboles en el bosque
    max_depth=15,       # Profundidad máxima de los árboles
    min_samples_split=5,  # Número mínimo de muestras requeridas para dividir un nodo interno
    min_samples_leaf=2,   # Número mínimo de muestras requeridas para ser un nodo hoja
    random_state=42      # Semilla para reproducibilidad
)

rf_model.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluamos el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Squared Error (MSE): 12.316446694545183
R-squared (R2): 0.9712029998130731
Mean Absolute Error (MAE): 1.070304946032503
Root Mean Squared Error (RMSE): 3.5094795475319676


In [15]:
# Calculamos la curva de aprendizaje
train_sizes, train_scores, test_scores = learning_curve(rf_model, X_train, y_train, cv=5)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

# Creamos el gráfico con Plotly
fig = go.Figure()

# Agregamos la curva de Training score
fig.add_trace(go.Scatter(x=train_sizes, y=train_scores_mean, mode='lines', name='Training Score'))

# Agregamos la curva de Cross-validation score
fig.add_trace(go.Scatter(x=train_sizes, y=test_scores_mean, mode='lines', name='Cross-validation Score'))

# Configuramos el diseño del gráfico
fig.update_layout(
    title='Curva de Aprendizaje',
    xaxis_title='Training examples',
    yaxis_title='Score',
    legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
)

# Mostramos el gráfico
fig.show()

In [16]:
# Creamos un DataFrame para Plotly Express
df = pd.DataFrame({'Valor Real': y_test, 'Predicciones': y_pred})

# Creamos gráfico de histograma apilado con Plotly Express
fig = px.histogram(df, nbins=30, marginal='rug', labels={'value': 'Valor'},
                   title='Distribución de Predicciones y Valores Reales',
                   category_orders={'variable': ['Valor Real', 'Predicciones']},
                   color_discrete_sequence=['blue', 'orange'],
                   barmode='overlay')

# Configuramos el diseño del gráfico
fig.update_layout(
    xaxis_title='Valor',
    yaxis_title='Frecuencia',
)

# Mostramos el gráfico
fig.show()

In [17]:
# Creamos un DataFrame para Plotly Express
df = pd.DataFrame({'Tipo': ['Valores Reales'] * len(y_test) + ['Predicciones'] * len(y_pred),
                   'Valor': list(y_test) + list(y_pred)})

# Creamos gráfico de violín con Plotly Express
fig = px.violin(df, x='Tipo', y='Valor', box=True, points="all", color='Tipo',
                labels={'Tipo': 'Tipo', 'Valor': 'Valor'},
                category_orders={'Tipo': ['Valores Reales', 'Predicciones']},
                violinmode='overlay')

# Configuramos el diseño del gráfico
fig.update_layout(
    title='Distribución de Predicciones y Valores Reales',
    xaxis_title='Tipo',
    yaxis_title='Valor',
    legend=dict(x=0.8, y=0.95),
)

# Mostramos el gráfico
fig.show()

## **XGBRegressor** 

#### **Aunque los resultados obtenidos con el método de Random Forest fueron alentadores, en el contexto de este proyecto se requiere una aproximación aún más cercana a la realidad. En este sentido, proponemos la implementación de un modelo XGBRegressor. Este enfoque busca no solo consolidar la calidad de las predicciones, sino también maximizar la precisión y la fidelidad del modelo en relación con los datos del mundo real.**

In [18]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializamos y entrenamos el modelo XGBRegressor
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# Evaluamos el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r2}')

Mean Squared Error (MSE): 1.820455824220891
Mean Absolute Error (MAE): 0.48914036629665664
Root Mean Squared Error (RMSE): 1.3492426854427972
R-squared (R2): 0.995743604627981


#### **Guardamos nuestro modelo**

In [19]:
# Guardamos el modelo en un archivo pickle
joblib.dump(xgb_model, 'xgb_model.pkl')

['xgb_model.pkl']

#### **Visualizamos los resultados**

In [20]:
# Creamos un DataFrame con las predicciones y valores reales
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Ordenamos el DataFrame por los valores reales para una mejor visualización
results_df = results_df.sort_values(by='Actual')

# Creamos gráfico de dispersión con Plotly Express
fig = px.scatter(results_df, x=results_df.index, y=['Actual', 'Predicted'],
                 labels={'value': 'Valor'},
                 title='Predicciones vs. Valores Reales',
                 color_discrete_sequence=['red', 'blue'])

# Configuramos el diseño del gráfico
fig.update_layout(
    xaxis_title='Índice',
    yaxis_title='Valor',
)

# Mostramos el gráfico
fig.show()

In [21]:
# Obtenemos la importancia de las características desde el modelo
feature_importance = xgb_model.feature_importances_

# Creamos un DataFrame con las características y su importancia
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# Ordenamos el DataFrame por importancia descendente
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Creamos gráfico de barras con Plotly Express
fig = px.bar(feature_importance_df, x='Feature', y='Importance',
             labels={'Importance': 'Importancia'},
             title='Importancia de Características',
             color='Importance',
             color_continuous_scale='Viridis')

# Agregamos líneas separadoras entre las barras (bordes negros)
fig.update_traces(marker_line_color='black', marker_line_width=1, opacity=0.7)

# Configuramos el diseño del gráfico
fig.update_layout(
    xaxis_title='Característica',
    yaxis_title='Importancia',
    xaxis=dict(tickangle=-45),
)

# Mostramos el gráfico
fig.show()

## **API**

In [3]:
# Importamos librerías
from fastapi import FastAPI, HTTPException, Body
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import joblib
import pandas as pd


# Instanciamos la aplicación
app = FastAPI()

# Cargar el modelo entrenado desde el archivo pickle
xgb_model = joblib.load('xgb_model.pkl')

class InputData(BaseModel):
    pBoroughID: float
    dayofweek: float
    hour: float

    # Agrega más campos según sea necesario

@app.post("/predict")
def predict(data: InputData):
    try:
        # Convertir los datos de entrada a un DataFrame
        input_data = pd.DataFrame([data.dict()])

        # Realizar la predicción
        prediction = xgb_model.predict(input_data)

        # Devolver la predicción como JSON
        return {"prediction": prediction.tolist()}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))